In [1]:
import os
import json
from django.db import IntegrityError
from django.core.exceptions import ObjectDoesNotExist


from functions.utils import get_data

ROOT_DIR = os.path.dirname(os.path.abspath('__file__'))
DATA_DIR = os.path.join(ROOT_DIR, 'data')

In [2]:
for dirpath, dirnames, filenames in os.walk(DATA_DIR):
    for filename in filenames:
        data = os.path.join(dirpath, filename)
        with open(data) as json_file:
            STORE = json.load(json_file)
                    
        
        # store 생성             
        try:
            store = Store.objects.get(url=STORE['url'])
        except ObjectDoesNotExist:
            store = Store.objects.create(
                url = get_data(STORE, ['url']),
                description = get_data(STORE, ['description']),
                name = get_data(STORE, ['name']),
                store_img = get_data(STORE, ['store_img']),
                is_delivery = get_data(STORE, ['is_delivery']),
                address = get_data(STORE, ['address']),
                lat = get_data(STORE, ['lat']),
                lng = get_data(STORE, ['lng']),
                food_type = get_data(STORE, ['store_type']),
                city = get_data(STORE, ['city'])
            )
            if get_data(STORE, ['delivery_fee']): store.delivery_fee = STORE['delivery_fee']
            if get_data(STORE, ['is_pickup']): store.is_pickup = STORE['is_pickup']
            if get_data(STORE, ['is_partner']): store.is_partner = STORE['is_partner']
            if get_data(STORE, ['last_order_time']):store.last_order_time = STORE['last_order_time']
            if get_data(STORE, ['estimated_prep_time']) : store.estimated_prep_time = STORE['estimated_prep_time']
            if get_data(STORE, ['delivery_message']) : store.delivery_message = STORE['delivery_message']
            
            store.save()
        
        # badge 생성
        if STORE['badges']:
            for badge in STORE['badges']:
                try:
                    Badge.objects.create(text=badge['text'])
                except IntegrityError:
                    pass
                finally:
                    B = Badge.objects.get(text=badge['text'])
                    store.badges.add(B)
                
        # open hours 생성
        if STORE['open_hours']:
            for open_hour in STORE['open_hours']:
                try:
                    oh = OpenHour.objects.create(store=store)
                except IntegrityError:
                    oh = OpenHour.objects.get(store=store)
                finally:
                    hour = Hour.objects.create(
                        open_hour = oh,
                        start = open_hour['start_time'],
                        end = open_hour['end_time'],
                        day = open_hour['day_of_week']
                    )
                    oh.hours.add(hour)

        for menu_category in get_data(STORE,['all_menus']):
            mc = MenuCategory.objects.create(
                store=store, 
                name=get_data(menu_category, ['category'])
            )
            
            for menu in get_data(menu_category, ['menus']):
                if get_data(menu, ['description']):
                    print(menu['description'].strip())
                    print("    >>>>>",menu['description'])
                # menu 생성
                M = Menu.objects.create(
                    name = get_data(menu, ['name']),
                    description = menu['description'].strip() if get_data(menu, ['description']) else None,
                    image_url = get_data(menu, ['img_url']),
                    price = get_data(menu, ['base_price'])
                )
                for option_category in get_data(menu, ['option_groups']):  
                    # option category 생성
                    try:
                        opt_cate = OptionCategory.objects.create(
                            name = get_data(option_category, ['category']),
                            is_required = get_data(option_category, ['is_required'])
                        )
                    except IntegrityError:
                        continue

                    # option 생성
                    for option in get_data(option_category, ['options']):
                        Option.objects.create(
                            category = opt_cate, 
                            name = get_data(option, ['name']), 
                            price = get_data(option, ['price'])
                        )
                    # menu에 option category 추가
                    M.option_categories.add(opt_cate)
                # menu category에 menu 추가
                mc.menu.add(M)
            

1 lb. product of japan
    >>>>> 1 lb. product of japan
6 or 10 pcs. big size chicken wings with cajun seasoning served with ranch and sweet sauce
    >>>>> 6 or 10 pcs. big size chicken wings with cajun seasoning served with ranch and sweet sauce
8 pcs. spinach & mozzarella with butter, herb and cheese on top
    >>>>> 8 pcs. spinach & mozzarella with butter, herb and cheese on top
8 pcs. spinach & mozzarella crispy served with cocktail sauce
    >>>>> 8 pcs. spinach & mozzarella crispy served with cocktail sauce
chicken
    >>>>> chicken
fresh vegetables home made
    >>>>> fresh vegetables home made
(not crab cake) crispy crab meat and chicken served with sweet & sour sauce
    >>>>> (not crab cake) crispy crab meat and chicken served with sweet & sour sauce
Sweet potato fries.
    >>>>> Sweet potato fries.
2 pieces crushed garlic with real butter
    >>>>> 2 pieces crushed garlic with real butter
hot & sour clear soup with shrimp, crawfish, fish filet, mussel, squid, live clam
    

Four fluffy, gluten-friendly pancakes topped with whipped real butter.
    >>>>> Four fluffy, gluten-friendly pancakes topped with whipped real butter.
Gluten-Friendly
    >>>>> Gluten-Friendly
Gluten-Friendly
    >>>>> Gluten-Friendly
Gluten-Friendly
    >>>>> Gluten-Friendly
Gluten-Friendly
    >>>>> Gluten-Friendly
Gluten-Friendly
    >>>>> Gluten-Friendly
Gluten-Friendly
    >>>>> Gluten-Friendly
Gluten-Friendly
    >>>>> Gluten-Friendly
Two fluffy, gluten-friendly pancakes topped with whipped real butter. Served with 2 eggs* your wa...
    >>>>> Two fluffy, gluten-friendly pancakes topped with whipped real butter. Served with 2 eggs* your wa...
A golden-brown, gluten-friendly Belgian waffle topped with whipped real butter. Served with 2 egg...
    >>>>> A golden-brown, gluten-friendly Belgian waffle topped with whipped real butter. Served with 2 egg...
A golden-brown, gluten-friendly Belgian waffle topped with whipped real butter.
    >>>>> A golden-brown, gluten-friendly Belgian 

Buttermilk crispy chicken breast tossed in Frank's RedHot® Buffalo sauce, lettuce, tomato, red on...
    >>>>> Buttermilk crispy chicken breast tossed in Frank's RedHot® Buffalo sauce, lettuce, tomato, red on...
Grilled sirloin steak & onions topped with melted American on a grilled roll.
    >>>>> Grilled sirloin steak & onions topped with melted American on a grilled roll.
Six strips of hickory-smoked bacon, lettuce, tomato, avocado & mayo on grilled sourdough.
    >>>>> Six strips of hickory-smoked bacon, lettuce, tomato, avocado & mayo on grilled sourdough.
Carved roasted turkey breast, White Cheddar, hickory-smoked bacon, lettuce, tomato & mayo on gril...
    >>>>> Carved roasted turkey breast, White Cheddar, hickory-smoked bacon, lettuce, tomato & mayo on gril...
Grilled artisan sourdough filled with scrambled eggs, sliced ham, Swiss & American.
    >>>>> Grilled artisan sourdough filled with scrambled eggs, sliced ham, Swiss & American.
Two steakburger patties, American and Whit

Made with freshly grilled chicken, pepper jack cheese, and your choice of tangy green sauce, chip...
    >>>>> Made with freshly grilled chicken, pepper jack cheese, and your choice of tangy green sauce, chip...
Freshly grilled marinated chicken, crispy fried tortilla strips, cilantro lime rice, and tangy ch...
    >>>>> Freshly grilled marinated chicken, crispy fried tortilla strips, cilantro lime rice, and tangy ch...
This crispy traditional Mexican donut is sprinkled with cinnamon sugar and is irresistible.
    >>>>> This crispy traditional Mexican donut is sprinkled with cinnamon sugar and is irresistible.
Freshly hand-grated cheddar cheese is folded in a flour tortilla and flat-grilled to perfection.
    >>>>> Freshly hand-grated cheddar cheese is folded in a flour tortilla and flat-grilled to perfection.
Crispy and sprinkled with Cinnamon Sugar and perfect for breakfast or a grab-and-go snack.
    >>>>> Crispy and sprinkled with Cinnamon Sugar and perfect for breakfast or a grab-

A Chicken Verde Wet Burrito, plus cilantro lime rice and beans & chips and salsa.
    >>>>> A Chicken Verde Wet Burrito, plus cilantro lime rice and beans & chips and salsa.
A Carne Asada Wet Burrito, plus cilantro lime rice and beans & chips and salsa.
    >>>>> A Carne Asada Wet Burrito, plus cilantro lime rice and beans & chips and salsa.
Our slow-cooked beans made from scratch are layered with cilantro lime rice, spicy jack cheese, a...
    >>>>> Our slow-cooked beans made from scratch are layered with cilantro lime rice, spicy jack cheese, a...
Slow-cooked beans made from scratch, zesty red sauce, and freshly hand-grated cheddar cheese all ...
    >>>>> Slow-cooked beans made from scratch, zesty red sauce, and freshly hand-grated cheddar cheese all ...
Slow-cooked beans made from scratch, tangy green sauce, and freshly hand-grated cheddar cheese al...
    >>>>> Slow-cooked beans made from scratch, tangy green sauce, and freshly hand-grated cheddar cheese al...
Seasoned beef with s

...

Our Double Whopper Sandwich is a pairing of two 1/4 lb* savory flame-grilled beef patties topped with thick-cut smoked bacon, American cheese, juicy tomatoes, fresh lettuce, creamy mayonnaise, ketchup, crunchy pickles, and sliced white onions on a soft sesame seed bun. Meal comes in medium and large sizes with a side of piping hot, thick cut French Fries and a fountain drink of your choice to make it a meal. *Based on pre­-cooked patty weight.
    >>>>> Our Double Whopper Sandwich is a pairing of two 1/4 lb* savory flame-grilled beef patties topped with thick-cut smoked bacon, American cheese, juicy tomatoes, fresh lettuce, creamy mayonnaise, ketchup, crunchy pickles, and sliced white onions on a soft sesame seed bun. Meal comes in medium and large sizes with a side of piping hot, thick cut French Fries and a fountain drink of your choice to make it a meal. *Based on pre­-cooked patty weight.
Our Triple Whopper Sandwich includes three 1/4 lb* savory flame-grilled beef patties topped wi

Minute Maid® Orange Juice explodes with flavor and is a good source of Vitamin C.© The Coca-Cola Company. Minute Maid" and "Put Good In. Get Good Out." are registered trademarks of The Coca-Cola Company."
    >>>>> Minute Maid® Orange Juice explodes with flavor and is a good source of Vitamin C.© The Coca-Cola Company. Minute Maid" and "Put Good In. Get Good Out." are registered trademarks of The Coca-Cola Company."
A good source of Calcium and Vitamin D, Fat Free Milk is a cool and refreshing complement to any meal.
    >>>>> A good source of Calcium and Vitamin D, Fat Free Milk is a cool and refreshing complement to any meal.
Velvety Low Fat Chocolate Milk delivers wholesome sweetness and is packed with Calcium.
    >>>>> Velvety Low Fat Chocolate Milk delivers wholesome sweetness and is packed with Calcium.
Nestlé® Pure Life® Purified Water, the exclusive bottled water of Burger King, is a cool and refreshing way to wash down your signature favorites. © Nestlé Waters North America I

Hot off the panini press! Comes with grilled chicken breast, pesto sauce, roasted mushrooms, house-grilled onions, and provolone cheese.
    >>>>> Hot off the panini press! Comes with grilled chicken breast, pesto sauce, roasted mushrooms, house-grilled onions, and provolone cheese.
Hot off the panini press! Comes with grilled chicken breast, crispy bacon, pesto sauce, roasted mushrooms, house-grilled onions, and provolone cheese.
    >>>>> Hot off the panini press! Comes with grilled chicken breast, crispy bacon, pesto sauce, roasted mushrooms, house-grilled onions, and provolone cheese.
Hot off the panini press! Comes with grilled chicken breast, chopped lettuce, onions, tomatoes, mozzarella cheese, mustard and mayo.
    >>>>> Hot off the panini press! Comes with grilled chicken breast, chopped lettuce, onions, tomatoes, mozzarella cheese, mustard and mayo.
Hot off the panini press! Comes with grilled chicken breast, marinara sauce, Parmesan cheese, and provolone cheese.
    >>>>> Ho

Fresh crispy French fries.
    >>>>> Fresh crispy French fries.
Thick cut onion rings are made from whole white onions, battered with a subtle blend of spiced and deep fried.
    >>>>> Thick cut onion rings are made from whole white onions, battered with a subtle blend of spiced and deep fried.
Crispy golden jalapeno stuffed with creamy cheese and served with savory marinara sauce.
    >>>>> Crispy golden jalapeno stuffed with creamy cheese and served with savory marinara sauce.
Fresh chicken wings marinated in a pepper seasoning and served with choice of dipping sauce.
    >>>>> Fresh chicken wings marinated in a pepper seasoning and served with choice of dipping sauce.
8 pieces crispy golden chicken nuggets and French fries served with choice of dipping sauce.
    >>>>> 8 pieces crispy golden chicken nuggets and French fries served with choice of dipping sauce.
5 pieces crispy golden white meat chicken tenders with choice of dipping sauce.
    >>>>> 5 pieces crispy golden white meat 

Spicy jalapenos topped with Italian sausage.
    >>>>> Spicy jalapenos topped with Italian sausage.
Pepperoni, mushrooms, onions, bell peppers and Italian sausage.
    >>>>> Pepperoni, mushrooms, onions, bell peppers and Italian sausage.
Creamy cheese with homemade tomato sauce.
    >>>>> Creamy cheese with homemade tomato sauce.
Creamy cheese and pepperoni with homemade tomato sauce.
    >>>>> Creamy cheese and pepperoni with homemade tomato sauce.
Hand cut french fries.
    >>>>> Hand cut french fries.
Crispy golden fried onion rings fresh out the fryer.
    >>>>> Crispy golden fried onion rings fresh out the fryer.
Golden crisp jalapeno poppers with melted mozzarella cheese and warm jalapeno.
    >>>>> Golden crisp jalapeno poppers with melted mozzarella cheese and warm jalapeno.
Mozzarella cheese sticks with marinara sauce.
    >>>>> Mozzarella cheese sticks with marinara sauce.
Crispy hot fried chicken wings.
    >>>>> Crispy hot fried chicken wings.
8 pieces golden chicken nugget